In [10]:
#####################################################################################################
#####################################################################################################
# 2_Import and cleaning abstracts
#####################################################################################################
#####################################################################################################

In [11]:
import os # File management
import nltk # Text management
from nltk.corpus import stopwords # Stopwords
import numpy # Mathematical operations
import matplotlib # Graphics
import string # Character chain management
from nltk.tokenize import word_tokenize
import pandas # Import data into DataFrame  
from sklearn.model_selection import train_test_split # Selection abstract for learning corpus  
from sklearn.feature_extraction.text import CountVectorizer # Matrix construction for learning
from sklearn.linear_model import LogisticRegression # Logistic regression
from sklearn import metrics # Confusion matrix
import pandas as pd

# Launch the program with all the functions
%run "C:\\Users\\....\\Program\\1_Functions.ipynb"

# Import of our 3 files, in a precise order
os.chdir(r"C:\\Users\\....\\Database\\")
abstract_embase=import_zot('Zot_Embase_1.txt')
abstract_coch=import_coch('Cochrane_2.txt')
abstract_pub=import_zot('Zot_Pubmed_3.txt')

# Creation of a complete abstract list
abstract_tot=abstract_embase+abstract_coch+abstract_pub


In [12]:
# We apply the function to our abstract list and we split it into words
doc_clean = [clean(doc).split() for doc in abstract_tot]      


In [13]:
# Import from UK / US dictionary
dic_uk_us = pd.read_excel (r'C:\\Users\\....\\Database\\dic_uk_us.xlsx')

# We transform the first column into a list
liste_uk=dic_uk_us['UK'].values.tolist()
liste_us=dic_uk_us['US'].values.tolist()

# We only keep the British orthography to homogenize
doc_clean_bis=[[liste_uk[liste_us.index(mot)] if mot in liste_us else mot for mot in a ] for a in doc_clean]

# We again clean up words that seem useless for further analysis
motsARetirer = ['background','icer','icers','purpose','p','n','per','also','mm','ci','thus','however','http','ii','iii','et','al','se','cc','email','eo','01','02','03','04','05','06','07','08','09']
doc_clean_nettoye = []

for sous_tableau in doc_clean_bis:
    tableau_nettoye = [mot for mot in sous_tableau if not mot in motsARetirer] 
    doc_clean_nettoye.append(tableau_nettoye)

# Gather all the words of the same abstract
doc_clean2=[]
for element in doc_clean_nettoye:
    mot_1=" ".join(element)
    doc_clean2.append(mot_1)

# Creation of a list with identifiers ranging from 1 to 3211
ident=[str(a) for a in range(1,3212,1)]

# We create a list with key / abstract pairs which we then transform into a dictionary
t=[]
for i in range(0,3211):
    t.append([ident[i],doc_clean2[i]])
    
# Dictionary creation
dict_abs=dict(t)
type(dict_abs)

# Import the xlsx file with the identifiers to keep
id_excel = pd.read_excel (r'C:\\Users\\....\\Database\\Identifier_to_import.xlsx', sheet_name='Second écrémage')

# We sort according to the identifier 1
id_excel=id_excel.sort_values('Identifier_1')

# We transform the first column into a list
liste_ident=id_excel['Identifier_1'].values.tolist()

# We transform into str
liste_ident=[str(i) for i in liste_ident] 

# Remove duplicates
article_conserve = {k: dict_abs[k] for k in liste_ident if k in dict_abs}

# Dictionary to dataframe transformation
base_articles=pd.DataFrame(list(article_conserve.items()), columns=['Identifier_1', 'Abstract'])

# We change the format of the binary column 0/1 and we merge it at the base of the articles (without duplicates) 
id_excel.Identifier_1=[str(i) for i in id_excel.Identifier_1] 
articles_vf=pd.merge(base_articles, id_excel, on=['Identifier_1']) 

# Delete articles with missing abstracts texts
articles_vf=articles_vf[articles_vf.Abstract != 'absent abstract']
len(articles_vf) 


2488